In [ ]:
# Test script for mf_toys
import sys, traceback
import nose
import unittest
from mfact import kpmf as mf
import torch
from torch.autograd import Variable
import numpy as np

In [ ]:
def test_init():
    test = [(500, 200, 100), (345, 123, 233), (120, 240, 29), (202, 800, 200)]
    for row, col, rank in test:
        model = mf.FactorModel(row, col, rank) # U and V are model weights
        
        # size of layer weights of U
        n_row, r = model.U.weight.data.size() 
        assert n_row == row
        assert r == rank
        
        # size of layer weights of V
        n_col, r = model.V.weight.data.size()
        assert n_col == col
        assert r == rank

In [ ]:
def test_forward():
    n_rows = 300
    n_cols = 20
    rank_actual = 2

    sigma = 0.000005

    # Covariance matrices
    row_info = np.random.random((n_rows, rank_actual))
    col_info = np.random.random((n_cols, rank_actual))
    row_info = Variable(torch.Tensor(row_info))
    col_info = Variable(torch.Tensor(col_info))

    k = mf.SEKernel()
    K_U = k(row_info, row_info)
    K_V = k(col_info, col_info)
    S_U = K_U.inverse()
    S_V = K_V.inverse()

    # targets, randomly generated vector (no need for U_actual, V_actual)
    targets = np.random.random_sample((128, )) # randomly generated numpy
    targets = Variable(torch.from_numpy(targets)) # convert to torch variable
    targets = targets.type(torch.FloatTensor) # convert to FloatTensor

    # preds, randomly generated vector from U and V (sim from model) - from forward
    model = mf.FactorModel(n_rows, n_cols, rank_actual)
    row = np.random.randint(5, size=(128, 1)) # random numpy rows for 1 batch 
    col = np.random.randint(5, size=(128, 1)) # random numpy cols for 1 batch
    row = Variable(torch.from_numpy(row)) # convert to torch variable
    col = Variable(torch.from_numpy(col)) # convert to torch variable
    preds = model(row, col) # forward to get predictions

    # Creating loss object
    loss_function = mf.KPMFLoss(sigma, K_U, K_V)

    # loss calculated by KPMFLoss in mf_toys
    loss = loss_function(preds, targets, model.U._parameters['weight'], model.V._parameters['weight'])

    # calculating loss using numpy

    # Likelihood, model weights U and V
    U = model.U._parameters['weight']
    V = model.V._parameters['weight']
    U_n = U.data.numpy()
    V_n = V.data.numpy()
    like = np.sum((preds.data.numpy() - targets.data.numpy()) ** 2)
    like /= 2.0 * sigma ** 2
    U_loss = 0
    for u in U_n.T:
        u = np.expand_dims(u, 0)
        U_loss += u @ S_U.data.numpy() @ u.T / 2
    U_loss = U_loss[0, 0]
    V_loss = 0
    for v in V_n.T:
        v = np.expand_dims(v, 0)
        V_loss += v @ S_V.data.numpy() @ v.T / 2
    V_loss = V_loss[0, 0]
    loss = np.array([L.data.numpy() for L in loss])
    assert np.isclose(like, loss[0])
    assert np.isclose(U_loss, loss[1])
    assert np.isclose(V_loss, loss[2])
    print(like, U_loss, V_loss)

In [ ]:
print('--> TEST_INIT')
test_init()
    
print('--> TEST_forward')
test_forward()